# Multi-Qubit Gates Workbook

This workbook describes the solutions to the problems offered in the "Multi-Qubit Gates" kata. Since the tasks are offered as programming problems, the explanations also cover some elements of Qiskit that might be non-obvious for a first-time user.

In [1]:
from qiskit import QuantumCircuit, QuantumRegister

## Exercise 1. Apply a tensor product of gates

One way to represent a multi-qubit transformation is to use the tensor product of gates acting on subsets of qubits.
Let's see how to reverse engineer the target matrix above to find the 3 gates which, acting on individual qubits, together form the target transformation.

Start by noticing that the top right and bottom left quadrants of the target matrix are filled with $0$'s, and the bottom right quadrant equals to the top left one, multiplied by $i$. This hints at applying the $S$ gate to the first qubit:

$$
Q =
\begin{bmatrix} 1 & 0 \\ 0 & i \end{bmatrix} \otimes
\begin{bmatrix}
    0 & -i & 0 & 0 \\ 
    i & 0 & 0 & 0  \\ 
    0 & 0 & 0 & -i \\ 
    0 & 0 & i & 0 
\end{bmatrix} =
\begin{bmatrix}
    0 & -i & 0 & 0 & 0 & 0 & 0 & 0 \\ 
    i & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 
    0 & 0 & 0 & -i & 0 & 0 & 0 & 0 \\ 
    0 & 0 & i & 0 & 0 & 0 & 0 & 0 \\ 
    0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\ 
    0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 \\ 
    0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\ 
    0 & 0 & 0 & 0 & 0 & 0 & -1 & 0
\end{bmatrix}
$$

Now the $4 \times 4$ matrix has all $0$s in the top right and bottom left quadrants, and the bottom right quadrant equals to the top left one. This means the second qubit has the $I$ gate applied to it, and the third qubit - the $Y$ gate:

$$
Q =
\begin{bmatrix} 1 & 0 \\ 0 & i \end{bmatrix} \otimes \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix} \otimes
\begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix} = S \otimes I \otimes Y
$$

> Remember to reverse the order of gates in the tensor product as you apply them to the Qiskit qubits: apply the $Y$ gate to `qr[0]` and the $S$ gate - to `qr[2]`.

In [ ]:
def apply_tensor_product(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.s(qr[2])
    circ.y(qr[0])

## Exercise 2. Prepare Bell state

You've seen this state before in the "Multi-Qubit Systems" kata, where you established that this state is not separable, that is, it can't be prepared using just the single-qubit gates. To prepare it, you need to use the CNOT gate.

Let's look at the effect of the CNOT gate on a separable state described in the tutorial:
$$\textrm{CNOT}_{1,2}\big(\alpha\ket{0} + \beta\ket{1}\big) \otimes \ket{0} = \textrm{CNOT}_{1,2}(\alpha\ket{00} + \beta\ket{10}) = \alpha\ket{00} + \beta\ket{11}$$

This resulting state is exactly the state you need to prepare, with $\alpha = \beta = \frac{1}{\sqrt{2}}$!

The solution takes two steps:
1. Prepare a state $\big(\frac{1}{\sqrt{2}}\ket{0} + \frac{1}{\sqrt{2}}\ket{1}\big) \otimes \ket{0}$.
You can use the Hadamard gate to do this.
2. Apply a CNOT gate with the first qubit as the control and the second qubit as the target.

> In this task, the order of qubits doesn't really matter, since the desired state is symmetrical. You just need to make sure that you're using the qubit you've prepared in a superposition state (the one you applied the Hadamard gate to) as the control for the CNOT gate.

In [ ]:
def prepare_bell_state(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.h(qr[0])
    circ.cx(qr[0], qr[1])

## Exercise 3. Entangle two qubits

Firstly, notice that you're dealing with an unentangled pair of qubits.
In vector form the transformation you need is
$$
\frac{1}{2}\begin{bmatrix}1 \\ 1 \\ 1 \\ 1 \end{bmatrix}
\rightarrow
\frac{1}{2}\begin{bmatrix}1 \\ 1 \\ 1 \\ -1 \end{bmatrix}
$$

All that needs to happen to change the input into the goal is that the $\ket{11}$ basis state needs to have its sign flipped.

Remember that the Pauli Z gate flips signs in the single qubit case, and that CZ is the 2-qubit version of this gate. And indeed, the effect of the CZ gate is exactly the transformation you're looking for here.

In [ ]:
def entangle_two_qubits(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.cz(qr[0], qr[1])

## Exercise 4. Swap two amplitudes

The SWAP gate described above allows you to swap the states of two qubits, which has the consequence of converting basis state $\ket{01}$ into $\ket{10}$ and vice versa without changing the basis states $\ket{00}$ and $\ket{11}$.

In [ ]:
def swap_amplitudes(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.swap(qr[0], qr[1])

## Exercise 5. Fredkin gate

The Fredkin gate is also known as the controlled SWAP gate. In big-endian notation, its matrix looks as follows:
$$
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
$$

If the initial state is written as a column vector in big-endian notation, it is
$$
\begin{bmatrix}
\alpha \\ \beta \\ \gamma \\ \delta \\ \epsilon \\ \zeta \\ \eta \\ \theta
\end{bmatrix}
$$
So you have:
$$
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
\alpha \\ \beta \\ \gamma \\ \delta \\ \epsilon \\ \color{blue}\zeta \\ \color{blue}\eta \\ \theta
\end{bmatrix} =
\begin{bmatrix}
\alpha \\ \beta \\ \gamma \\ \delta \\ \epsilon \\ \color{red}\eta \\ \color{red}\zeta \\ \theta
\end{bmatrix} =
\alpha \ket{000} + \beta \ket{001} + \gamma \ket{010} + \delta \ket{011} + \epsilon \ket{100} + {\color{red}\eta}\ket{101} + {\color{red}\zeta}\ket{110} + \theta\ket{111}
$$

When implementing this in Qiskit, you can use the built-in method `cswap`:

In [ ]:
def fredkin_gate(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.cswap(qr[0], qr[1], qr[2])

## Exercise 6. Controlled rotation

Qiskit's `cry` method applies exactly the gate you're looking for.

In [ ]:
def controlled_rotation(circ: QuantumCircuit, qr: QuantumRegister, theta: float) -> None:
    circ.cry(theta, qr[0], qr[1])

## Exercise 7. Controlled phase gate

The phase gate is implemented as `PhaseGate` class. Its constructor takes one argument, the angle $theta$. Calling the `control` method creates a three-qubit gate that acts exactly as described in the exercise.

In [ ]:
def controlled_phase(circ: QuantumCircuit, qr: QuantumRegister, theta: float) -> None:
    from qiskit.circuit.library import PhaseGate
    cphase = PhaseGate(theta).control(2)
    circ.append(cphase, qr)

## Exercise 8. Anti-controlled gate

You can achieve the goal by applying of a Pauli X gate on the second qubit when the first qubit is in the $\ket{0}$ state. This is exactly the controlled-on-zero X gate.

In [ ]:
def anti_controlled_gate(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.cx(qr[0], qr[1], ctrl_state=0)